## Q1

In [40]:
from fastembed.embedding import TextEmbedding
import numpy as np

# Load model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Embed query (note: returns a generator)
query = "I just discovered the course. Can I join now?"

embedding_query = list(model.embed([query]))[0]  # Convert from generator
embedding_query_array = np.array(embedding_query)

# Print shape and min value
print("Shape:", embedding_query_array.shape)
print("Min value:", round(np.min(embedding_query_array), 2))

Shape: (512,)
Min value: -0.12


## Q2

In [41]:
doc = 'Can I still join the course after the start date?'

embedding_doc = list(model.embed([doc]))[0]  # Convert from generator
embedding_doc_array = np.array(embedding_doc)
embedding_query_array.dot(embedding_doc_array)

np.float64(0.9008528895674548)

## Q3

In [42]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

cosine = -1
res = []
for i, document in enumerate(documents):
    doc = document['text']
    embedding_doc = list(model.embed([doc]))[0]  # Convert from generator
    embedding_doc_array = np.array(embedding_doc)
    cosine = embedding_query_array.dot(embedding_doc_array), cosine
    res.append([cosine, i])
res

[[(np.float64(0.7629684696540238), -1), 0],
 [(np.float64(0.8182378150042889), (np.float64(0.7629684696540238), -1)), 1],
 [(np.float64(0.8085397398734189),
   (np.float64(0.8182378150042889), (np.float64(0.7629684696540238), -1))),
  2],
 [(np.float64(0.7133079015686243),
   (np.float64(0.8085397398734189),
    (np.float64(0.8182378150042889), (np.float64(0.7629684696540238), -1)))),
  3],
 [(np.float64(0.7304499234333598),
   (np.float64(0.7133079015686243),
    (np.float64(0.8085397398734189),
     (np.float64(0.8182378150042889), (np.float64(0.7629684696540238), -1))))),
  4]]

In [43]:
V = np.array([list(model.embed([doc['text']]))[0] for doc in documents])  # Shape: (5, d)
cosine_similarities = V.dot(embedding_query_array)  # Shape: (5,)
most_similar_index = np.argmax(cosine_similarities)
most_similar_index

np.int64(1)

## Q4

In [44]:
V = np.array([list(model.embed([doc['question'] + ' ' + doc['text']]))[0] for doc in documents])  # Shape: (5, d)
cosine_similarities = V.dot(embedding_query_array)  # Shape: (5,)
most_similar_index = np.argmax(cosine_similarities)
most_similar_index

np.int64(0)

## Q5

In [48]:
from fastembed import TextEmbedding

import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model['dim'] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

## Q6

In [56]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import requests
import numpy as np

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

model_handle = "BAAI/bge-small-en"
embedder = TextEmbedding(model_name=model_handle)
embedding_dim = 384 

client = QdrantClient("http://localhost:6333")
collection_name = "zoomcamp-assignment2"

client.delete_collection(collection_name=collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_dim,
        distance=models.Distance.COSINE
    )
)

points = []
for idx, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    embedding = list(embedder.embed([full_text]))[0]

    point = models.PointStruct(
        id=idx,
        vector=embedding,
        payload={
            "text": doc['text'],
            "question": doc['question'],
            "course": doc['course']
        }
    )
    points.append(point)

client.upsert(collection_name=collection_name, points=points)

def search(query, limit=5):
    query_embedding = list(embedder.embed([query]))[0]
    results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=limit,
        with_payload=True
    )
    return results

query = "I just discovered the course. Can I join now?"
results = search(query)

print(results)
print(f"Top score: {results[0].score:.2f}")


[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=13, version=0, score=0.86833113, payload={'text': "Here’s how you join a in

/tmp/ipykernel_3440/2409213519.py:54: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
